# Run Talos hyperparamater tuning

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa

from IPython.display import display, Markdown
plt.style.use('seaborn-darkgrid')
pd.set_option('display.max_columns', None) 

DEBUG = False

## Setup Colab

In [2]:
COLAB = 'google.colab' in str(get_ipython())

if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

## Load local modules

In [3]:
import os
import sys

if not COLAB:
    ROOT = os.path.join(os.getcwd(), '..', '..') 

    src_dir = os.path.join(ROOT, 'src')
    sys.path.append(src_dir)

    from constants import *
else:
    N_MELS = 80
    CONTEXT_WINDOW = 17
    SUB_DIVISION = 4

## Set random seed for reproducability

In [ ]:
np.random.seed(42)

## Load TensorFlow modules

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Install and load required dependencies

In [6]:
import talos
import pickle5 as pickle

## Enable TPU

In [7]:
if ('COLAB_TPU_ADDR' in os.environ.keys()):
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_host(resolver.master())
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

## Load dataset

In [9]:
path = os.path.join(ROOT, SUB_DIVS_DIR, 'beats', 'harmonix.p') if not COLAB else '/content/drive/MyDrive/fyp/collabs/harmonix.p'
harmonix_beats = pickle.load(open(path, 'rb'))
harmonix_beats = harmonix_beats.head(50)
harmonix_beats.head()

,File,Sub_Divisions,Binary_Labels,Weighted_Labels,Weights,IDS,Beat_times,Labels,BPM
0,0001_12step,[[[[-1.09821814 -1.1065893 -1.10661448 -1.106...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0001_12step, 0001_12step, 0001_12step, 0001_1...","[0.0, 0.5309729999999999, 1.0619459999999998, ...","[0.0, 8.495567999999999, 25.486704, 42.4753280...",113
1,0003_6foot7foot,[[[[-1.53216075 -1.5321569 -1.53216468 -1.532...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, ...","[0003_6foot7foot, 0003_6foot7foot, 0003_6foot7...","[2.857108, 3.571394, 4.28568, 4.99996600000000...","[2.857108, 8.571396, 31.428548, 37.14283599999...",84
2,0004_abc,[[[[-1.28460321 -1.29111018 -1.29454499 -1.302...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0004_abc, 0004_abc, 0004_abc, 0004_abc, 0004_...","[2.666656, 3.238084, 3.952369, 4.597529, 5.242...","[2.666656, 28.300542999999998, 58.263180000000...",94
3,0006_aint2proud2beg,[[[[-1.47435298 -1.48497827 -1.49480312 -1.497...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0006_aint2proud2beg, 0006_aint2proud2beg, 000...","[0.0, 0.572203, 1.144406, 1.716609, 2.288812, ...","[0.0, 27.4652, 45.203726, 63.518522999999995, ...",105
4,0008_america,[[[[-1.25359642 -1.25824786 -1.26322129 -1.265...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0008_america, 0008_america, 0008_america, 000...","[3.871208, 4.359011, 4.846814, 5.338616, 5.830...","[3.871208, 10.56504, 33.217138, 56.85190400000...",136


## Splits

In [10]:
X = harmonix_beats
y = harmonix_beats['Binary_Labels']

In [11]:
X_train, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, _, _ = train_test_split(X_train, X_train['Binary_Labels'], test_size=0.1, random_state=42)

In [12]:
y_train = np.concatenate(X_train['Weighted_Labels'].values)
y_test = np.concatenate(X_test['Binary_Labels'].values)
y_val = np.concatenate(X_val['Binary_Labels'].values)

ids_train = np.concatenate(X_train['IDS'].values)
ids_test = np.concatenate(X_test['IDS'].values)
ids_val = np.concatenate(X_val['IDS'].values)

w_train = np.concatenate(X_train['Weights'].values)
w_test = np.concatenate(X_test['Weights'].values)
w_val = np.concatenate(X_val['Weights'].values)

X_train = np.concatenate(X_train['Sub_Divisions'].values)
X_test = np.concatenate(X_test['Sub_Divisions'].values)
X_val = np.concatenate(X_val['Sub_Divisions'].values)

In [13]:
X_train = X_train.swapaxes(2,3)
X_test = X_test.swapaxes(2,3)
X_val = X_val.swapaxes(2,3)

## Prepare Tuning

### Set the grid parameters

In [14]:
params = {
    'first_neuron': [8, 16, 32],
    'second_neuron': [16, 32, 64],
    'third_neuron': [64, 128, 256],
    'dropout_one': [0.3, 0.4, 0.5],
    'dropout_two': [0.3, 0.4, 0.5],
    'dropout_three': [0.2, 0.3, 0.4, 0.5],
    'activation':['elu', 'relu'],
    'optimizer': ['Adam', 'SGD'],
    'batch_size': [24, 32],
    'learning_rate': (0.1, 10, 10),
    'epochs': [80, 100],
    'w_train': [w_train]
}

### Metrics loader helper

In [15]:
def load_metrics():
    return [
        tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.15),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
    ]

### Build the model

In [16]:
def load_model(params):
    initializer = tf.keras.initializers.HeNormal()
    return Sequential([
        Input(input_shape=(N_MELS, CONTEXT_WINDOW, SUBDIVISIONS)),
        Conv2D(params['first_neuron'], (8,6), activation=params['activation'], kernel_initializer=initializer, padding="same"),
        MaxPooling2D(pool_size=(5, 2)),
        BatchNormalization(),
        Dropout(params['dropout_one']),
        Conv2D(params['second_neuron'], (6,4), activation=params['activation'], padding="same", kernel_initializer=initializer),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Dropout(params['dropout_one']),
        Flatten(),
        Dense(params['third_neuron'], activation=params['activation']),
        Dropout(params['dropout_two']),
        Dense(1, activation='sigmoid')
    ])

### Fit the model

In [17]:
def fit_model(X_train, y_train, X_val, y_val, params):
    return model.fit(
        X_train, 
        y_train, 
        batch_size=params['batch_size'],
        epochs=params['epochs'], 
        shuffle=True,
        verbose=0,
        sample_weight=params['w_train'],
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

### Compile the model

In [18]:
def model_tuning(X_train, y_train, X_val, y_val, params):
    load_metrics()
    model = build_model()
    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'], metrics=METRICS)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=20, mode='max', restore_best_weights=True)
    history = fit_model() 
    return history, model

### Run the hyperparameter tuning

In [19]:
## Only run in colab
if ('COLAB_TPU_ADDR' in os.environ.keys()):
    with strategy.scope():
        scan_object = talos.Scan(
            X_train,
            y_train, 
            params=params,
            model=model_tuning,
            experiment_name='nnssa',
            fraction_limit=.001,
            x_val = X_val,
            y_val = y_val
        )